In [1]:

from src.hamiltonian_utils import FermiHubbardHamiltonian
from src.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState,QuadrupoleOperator,J2operator
import numpy as np
import torch
from typing import Dict
import scipy
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
from src.utils_quasiparticle_approximation import QuasiParticlesConverter,HardcoreBosonsBasis
from scipy.sparse import lil_matrix
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=2
nparticles_b=2

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{18}$O'

In [2]:
matrix,_=get_twobody_nuclearshell_model(file_name=file_name)

Computing the matrix, pls wait... (u_u) 



100%|██████████| 24/24 [00:45<00:00,  1.88s/it]


In [3]:
print(matrix)

{(0, 1, 0, 1): np.float64(-0.20690000000000003), (0, 1, 1, 0): np.float64(0.20690000000000003), (1, 0, 0, 1): np.float64(0.20690000000000003), (1, 0, 1, 0): np.float64(-0.20690000000000003), (0, 1, 0, 8): np.float64(1.3349000000000004), (0, 1, 8, 0): np.float64(-1.3349000000000004), (1, 0, 0, 8): np.float64(-1.3349000000000004), (1, 0, 8, 0): np.float64(1.3349000000000004), (0, 2, 0, 2): np.float64(-0.20690000000000003), (0, 2, 2, 0): np.float64(0.20690000000000003), (2, 0, 0, 2): np.float64(0.20690000000000003), (2, 0, 2, 0): np.float64(-0.20690000000000003), (0, 2, 0, 9): np.float64(0.8174559644103165), (0, 2, 9, 0): np.float64(-0.8174559644103165), (2, 0, 0, 9): np.float64(-0.8174559644103165), (2, 0, 9, 0): np.float64(0.8174559644103165), (0, 2, 1, 8): np.float64(1.0553311121396927), (0, 2, 8, 1): np.float64(-1.0553311121396927), (2, 0, 1, 8): np.float64(-1.0553311121396927), (2, 0, 8, 1): np.float64(1.0553311121396927), (0, 3, 0, 3): np.float64(-0.49040000000000067), (0, 3, 3, 0):

In [4]:
print(matrix[(12,13,12,13)])

-0.20689999999999995


In [6]:
print(matrix[(1,16,2,19)])

KeyError: (1, 16, 2, 19)

In [5]:
# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])

NSMHamiltonian.get_twobody_interaction(twobody_dict=matrix)

NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)
print(NSMHamiltonian.basis)
print(NSMHamiltonian.hamiltonian)
print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis))

size= 24 12


9648it [01:12, 132.73it/s]


[-39.44932325]
[[1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 1]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 30656 stored elements and shape (640, 640)>
  Coords	Values
  (0, 0)	-21.637774603174606
  (0, 1)	2.1441927358413584
  (0, 2)	-0.10915079365079428
  (0, 3)	-0.6398020847719197
  (0, 5)	0.5211067201086399
  (0, 6)	-0.9690507936507938
  (0, 7)	0.22802569399122857
  (0, 10)	0.45225308401658526
  (0, 12)	-1.2129704215656378
  (0, 15)	-0.5542253968253976
  (0, 16)	0.43082268503532073
  (0, 21)	0.2890940607855491
  (0, 22)	-0.2239312965819814
  (0, 26)	-0.45876507936507965
  (0, 27)	0.4680380952380956
  (0, 34)	1.096283198521862
  (0, 42)	-0.5858860751129027
  (0, 47)	-0.20198421437613098
  (0, 49)	0.3148958148767512
  (0, 53)	2.1441927358413597
  (0, 54)	-0.3039641477372983
  (0, 55)	0.5211067201086399
  (0, 56)	0.1340990266621962
  (0, 58)	0.15358775946623415
  (0, 59)	-1.2129704215656374
  :	:
 